# Laboratório 2: Expansão de consultas usando word2vec

## Introdução

Nesta atividade você vai exercitar a noção de expansão de consultas. Considerando a coleção de notícias do lab passado, você primeiro precisa executar os seguintes passos:

Escreva uma função que receba uma coleção de documentos e retorne uma matrix de termos-termos contendo as frequências de co-ocorrência de duas palavras consecutivas no texto (bigramas).
Escreva uma função que receba um certo termo de consulta e a matriz construída no passo 1 acima e retorneas top-3 palavras em ordem decrescente de frequencia.
Expanda a consulta original com os termos retornados no passo 2 acima.
Faça uma busca disjuntiva (OR) considerando a nova consulta.
Por exemplo, se o termo de consulta é petrobrás e as palavras que mais co-ocorrem com petrobrás são: empresa, gasolina e estatal, a consulta agora ficaria: petrobrás OR empresa OR gasolina OR estatal.

Escolha livremente três termos de consulta e responda o seguinte:

Quais os termos retornados para a expansão de cada consulta?
Você acha que esses termos são de fato relacionados com a consulta original? Justifique.
Compare os documentos retornados para a consulta original com a consulta expandida. Quais resultados você acha que melhor capturam a necessidade de informação do usuário? Por que?
A expansão de consultas é mais adequada para melhorar o recall ou o precision? Por que?

## Preparando dataset e _imports_ 

Para essa análise utilizaremos as bibliotecas costumeiras, tais como _pandas_, _numpy_ e _nltk_. Além disso, vamos utilizar o modelo _word2vec_ para montar nossa matriz de ocorrência, de forma que utilizaremos a biblioteca _gensim_. 

In [1]:
import pandas as pd
import numpy as np
import nltk as nl
import gensim as gs
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from gensim import corpora, models, similarities

Nosso _dataset_ é o conjunto de notícias do Estadão contendo _timestamp_, _titulo_, _subTitulo_,_conteudo_,_url_ e _idNoticia_.

In [67]:
df = pd.read_csv("../database/estadao_noticias_eleicao.csv", encoding='utf-8')
df = df.replace(np.nan, '', regex = True)
content = df.titulo + " " + df.conteudo
content = content.fillna("")
df.head()
df.iloc[[5791]]

,timestamp,titulo,subTitulo,conteudo,url,idNoticia
5791,2014-06-10T00:00:00Z,"Com 59% dos votos válidos, PMDB confirma alian...",Decisão confirma o nome de Michel Temer como p...,"Brasília - Por maioria, o PMDB decidiu , na ta...","http://politica.estadao.com.br/noticias/geral,...",5792


## Criando a matriz de ocorrência usando _gensim_

## Criando os tokens e vocabulário

In [3]:
tokenizer = RegexpTokenizer(r'\w+')
tokens_lists = content.apply(lambda text: tokenizer.tokenize(text.lower()))

stopword_ = stopwords.words('portuguese')
filtered_tokens = tokens_lists.apply(lambda tokens: [token for token in tokens if token not in stopword_])

In [4]:
#tokens = [token for tokens_list in filtered_tokens for token in tokens_list]

In [5]:
dictionary = corpora.Dictionary(filtered_tokens)

In [6]:
vec = [(0, 1), (4, 1)]

In [7]:
corpus = [dictionary.doc2bow(text) for text in filtered_tokens]

In [8]:
lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=2)

In [53]:
doc = "Lula"
vec_bow = dictionary.doc2bow(doc.lower().split())
vec_lsi = lsi[vec_bow] # convert the query to LSI space
print(vec_lsi)

[(0, 0.0786814501333691), (1, -0.10397658312009153)]


In [55]:
index = similarities.MatrixSimilarity(lsi[corpus])

/home/paulo.soares/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [57]:
sims = index[vec_lsi]
sims = sorted(enumerate(sims), key=lambda item: -item[1])
print(sims)
print(dictionary[sims[1190][0]])
print(dictionary[sims[2791][0]])
print(dictionary[sims[2104][0]])

empresa
ma
gerência


## Escrevendo a função de busca

In [ ]:
df